# Second Test

I try to predict the subtask using a fixed set with subtask. But I think that's is not the best.

In [1]:
from huggingface_hub import login
login()

In [2]:
!uv pip install "lerobot[pi] @ git+https://github.com/huggingface/lerobot.git"

Using Python 3.12.12 environment at: /usr
Resolved 108 packages in 922ms
Audited 108 packages in 1ms


In [3]:
import math
import torch
from lerobot.policies.pi05.modeling_pi05 import PI05Policy
from transformers import AutoProcessor
from PIL import Image

In [9]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

In [10]:
!ls

sample_data  scene.jpg


In [ ]:
import json
import draccus
from huggingface_hub import hf_hub_download

# 1) Load config manually, remove 'type' field, and force CPU
# This bypasses the DecodingError and prevents immediate CUDA loading
config_path = hf_hub_download("lerobot/pi05_base", "config.json")
with open(config_path, "r") as f:
    cfg_dict = json.load(f)

# The 'type' field is present in the json but not in PI05Config dataclass, causing strict parsing to fail
if "type" in cfg_dict:
    del cfg_dict["type"]

config = draccus.decode(PI05Policy.config_class, cfg_dict)
config.device = "cpu"  # Force CPU loading

policy = PI05Policy.from_pretrained(
    "lerobot/pi05_base",
    config=config,
).eval()

paligemma = policy.model.paligemma_with_expert.paligemma

# 2) Use the matching processor
processor = AutoProcessor.from_pretrained("google/paligemma-3b-pt-224")

The PI05 model is a direct port of the OpenPI implementation. 
This implementation follows the original OpenPI structure for compatibility. 
Original implementation: https://github.com/Physical-Intelligence/openpi
Loading model from: lerobot/pi05_base


✓ Loaded state dict from model.safetensors
Remapped: action_in_proj.bias -> model.action_in_proj.bias
Remapped: action_in_proj.weight -> model.action_in_proj.weight
Remapped: action_out_proj.bias -> model.action_out_proj.bias
Remapped: action_out_proj.weight -> model.action_out_proj.weight
Remapped: paligemma_with_expert.gemma_expert.lm_head.weight -> model.paligemma_with_expert.gemma_expert.lm_head.weight
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.bias -> model.paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.bias
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.weight -> model.paligemma_with_expert.gemma_expert.model.layers.0.input_layernorm.dense.weight
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.mlp.down_proj.weight -> model.paligemma_with_expert.gemma_expert.model.layers.0.mlp.down_proj.weight
Remapped: paligemma_with_expert.gemma_expert.model.layers.0.mlp.gate_proj.weigh

In [12]:
# --- FIX: Resize embeddings on CPU ---
tokenizer_vocab_size = len(processor.tokenizer)
model_vocab_size = paligemma.get_input_embeddings().weight.shape[0]

print(f"Vocab sizes - Tokenizer: {tokenizer_vocab_size}, Model: {model_vocab_size}")
if model_vocab_size < tokenizer_vocab_size:
    print(f"Resizing model embeddings to {tokenizer_vocab_size}...")
    paligemma.resize_token_embeddings(tokenizer_vocab_size)
# -------------------------------------

# 3) Now safe to move to GPU
print(f"Moving model to {DEVICE}...")
policy.to(DEVICE)
paligemma = policy.model.paligemma_with_expert.paligemma # Update reference

# (Optional tie weights)
if hasattr(paligemma, "tie_weights"):
    print("Tying weights...")
    paligemma.tie_weights()
try:
    paligemma.model.language_model.embed_tokens.weight = paligemma.lm_head.weight
except Exception:
    pass

Vocab sizes - Tokenizer: 257153, Model: 257152
Resizing model embeddings to 257153...
Moving model to cuda...


In [ ]:
@torch.no_grad()
def score_text(image: Image.Image, prompt: str, completion: str) -> float:
    """
    Returns log P(completion | image, prompt) using teacher forcing.
    """
    # PaliGemma wants <image> in the text
    full = f"<image>\n{prompt}{completion}"
    inputs = processor(text=full, images=image, return_tensors="pt")
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    # Labels = input_ids shifted for LM loss
    input_ids = inputs["input_ids"]
    labels = input_ids.clone()

    out = paligemma(**inputs, labels=labels)
    print("OUT", type(out))
    n_tokens = input_ids.numel()
    print("N_TOKENS", type(out))
    return float(-out.loss * n_tokens)


@torch.no_grad()
def pick_subtask(image_path: str, high_level_task: str, candidates: list[str]) -> tuple[str, float]:
    image = Image.open(image_path).convert("RGB")
    prompt = f"Task: {high_level_task}\nNext subtask: "

    best = None
    best_score = -math.inf
    for c in candidates:
        s = score_text(image, prompt, c)
        if s > best_score:
            best_score = s
            best = c
    return best, best_score

In [24]:
candidates = [
        "pick up the pillow",
        "adjust the blanket",
        "pick up the plate",
        "put the plate in the sink",
        "open the drawer",
    ]
subtask, score = pick_subtask(
        image_path="scene.jpg",
        high_level_task="pillow",
        candidates=candidates,)
print("predicted subtask:", subtask)
print("score:", score)

OUT <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
N_TOKENS <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
OUT <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
N_TOKENS <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
OUT <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
N_TOKENS <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
OUT <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
N_TOKENS <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
OUT <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
N_TOKENS <class 'transformers.models.paligemma.modeling_paligemma.PaliGemmaCausalLMOutputWithPast'>
predicted subtask: pick u